# Distributed Data Parallel EfficientNet Training with PyTorch and SageMaker Distributed

[Amazon SageMaker's distributed library](https://docs.aws.amazon.com/sagemaker/latest/dg/distributed-training.html) can be used to train deep learning models faster and cheaper. The [data parallel](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel.html) feature in this library (`smdistributed.dataparallel`) is a distributed data parallel training framework for PyTorch, TensorFlow, and MXNet.

This notebook demonstrates how to use `smdistributed.dataparallel` with PyTorch on [Amazon SageMaker](https://aws.amazon.com/sagemaker/) to train an EfficientNet model on a large image dataset such as [ImageNet](https://image-net.org/download.php) using [Amazon FSx for Lustre file-system](https://aws.amazon.com/fsx/lustre/) as data source.

The outline of steps is as follows:

1. Stage the ImageNet dataset files on [Amazon S3](https://aws.amazon.com/s3/)
2. Create Amazon FSx Lustre file-system and import data into the file-system from S3
3. Build Docker training image and push it to [Amazon ECR](https://aws.amazon.com/ecr/)
4. Configure data input channels for SageMaker
5. Configure hyper-prarameters
6. Define training metrics
7. Define training job, set distribution strategy to SMDataParallel and start training

**NOTE:** With large training dataset such as ImageNet, we recommend using [Amazon FSx](https://aws.amazon.com/fsx/) as the input file system for the SageMaker training job. FSx file input to SageMaker significantly cuts down training start up time on SageMaker because it avoids downloading the training data each time you start the training job (as done with S3 input for SageMaker training job) and provides good data read throughput.


**NOTE:** This example requires SageMaker Python SDK v2.X.

## Amazon SageMaker Initialization

Initialize the notebook instance. Get the AWS Region and a SageMaker execution role.

### SageMaker role

The following code cell defines `role` which is the IAM role ARN used to create and run SageMaker training and hosting jobs. This is the same IAM role used to create this SageMaker Notebook instance. 

`role` must have permission to create a SageMaker training job and host a model. For granular policies you can use to grant these permissions, see [Amazon SageMaker Roles](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html). If you do not require fine-tuned permissions for this demo, you can use the IAM managed policy AmazonSageMakerFullAccess to complete this demo. 

As described above, since we will be using FSx, please make sure to attach `FSx Access` permission to this IAM role. If you do not require fine-tuned permissions for this demo, you can use the IAM managed policy AmazonFSxFullAccess to complete this demo.

In [ ]:
%%time
! python3 -m pip install --upgrade sagemaker
import sagemaker
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import boto3

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
role_name = role.split(["/"][-1])
print(f"SageMaker Execution Role: {role}")
print(f"The name of the Execution role: {role_name[-1]}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

To verify that the role above has required permissions:

1. Go to the IAM console: https://console.aws.amazon.com/iam/home.
2. Select **Roles**.
3. Enter the role name in the search box to search for that role. 
4. Select the role.
5. Use the **Permissions** tab to verify this role has required permissions attached.

## Prepare SageMaker Training Images

1. SageMaker by default use the latest [Amazon Deep Learning Container Images (DLC)](https://github.com/aws/deep-learning-containers/blob/master/available_images.md) PyTorch training image. In this step, we use it as a base image and install additional dependencies required for training EfficientNet model.
2. In this [GitHub repository](https://github.com/HerringForks/SMDDP-Examples/tree/main/pytorch/efficientnet), we have forked an EfficientNet example from [NVIDIA/DeepLearningExamples](https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Classification/ConvNets) and adapted the training script to work with `smdistributed.dataparallel`.
3. Clone the repository and copy all files to the docker workspace.

### Build and Push Docker Image to ECR

Run the below command build the docker image and push it to ECR.

In [ ]:
image = "pt-smdataparallel-efficientnet-sagemaker"  # Example: pt-smdataparallel-efficientnet-sagemaker
tag = "latest"  # Example: latest

In [ ]:
!pygmentize ./Dockerfile

In [ ]:
!pygmentize ./build_and_push.sh

In [ ]:
# Clone herring forks repository for reference implementation EfficientNet with PyTorch-SMDataParallel
!rm -rf SMDDP-Examples
!git clone --recursive https://github.com/HerringForks/SMDDP-Examples.git
    
# Build and push the container
! aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin 763104351884.dkr.ecr.{region}.amazonaws.com
! chmod +x build_and_push.sh; bash build_and_push.sh {region} {image} {tag}

## Preparing FSx Input for SageMaker

1. Download, prepare, and upload your training dataset on Amazon S3. Follow these [steps to download and convert the ImageNet dataset to TFRecords format](https://github.com/kmonachopoulos/ImageNet-to-TFrecord).
2. Follow these [steps to create a FSx linked with your S3 bucket with training data](https://docs.aws.amazon.com/fsx/latest/LustreGuide/create-fs-linked-data-repo.html). Make sure to add an endpoint to your VPC allowing S3 access.
3. Follow these [steps to configure your SageMaker training job to use FSx](https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/).

### Important Caveats

1. You need to use the same `subnet` and `vpc` and `security group` used with FSx when launching the SageMaker notebook instance. The same configurations will be used by your SageMaker training job.
2. Make sure you set the [appropriate inbound/output rules in the `security group`](https://docs.aws.amazon.com/fsx/latest/LustreGuide/limit-access-security-groups.html). Specifically, opening up these ports is necessary for SageMaker to access the FSx filesystem in the training job.
3. Make sure `SageMaker IAM Role` used to launch this SageMaker training job has access to `AmazonFSx`.

## SageMaker PyTorch Estimator function options

In the following code block, you can update the estimator function to use a different instance type, instance count, and distribution strategy. You're also passing in the training script you reviewed in the previous cell.

**Instance types**

`smdistributed.dataparallel` supports model training on SageMaker with the following instance types only. For best performance, it is recommended you use an instance type that supports Amazon Elastic Fabric Adapter (ml.p3dn.24xlarge and ml.p4d.24xlarge).

1. ml.p3.16xlarge
1. ml.p3dn.24xlarge [Recommended]
1. ml.p4d.24xlarge [Recommended]

**Instance count**

To get the best performance and the most out of `smdistributed.dataparallel`, you should use at least 2 instances, but you can also use 1 for testing this example.

**Distribution strategy**

Note that to use DDP mode, you need to update the `distribution` strategy, and set it to use `smdistributed dataparallel`.

In [ ]:
from sagemaker.pytorch import PyTorch

In [ ]:
instance_type = "ml.p4d.24xlarge"  # Other supported instance type: ml.p3.16xlarge, ml.p3dn.24xlarge
instance_count = 2  # You can use 2, 4, 8 etc.
docker_image = f"{account}.dkr.ecr.{region}.amazonaws.com/{image}:{tag}"  # YOUR_ECR_IMAGE_BUILT_WITH_ABOVE_DOCKER_FILE
username = "AWS"
subnets = ["<subnet_id>"]  # Should be same as Subnet used for FSx. Example: subnet-0f9XXXX
security_group_ids = [
    "<security_group_id>"
]  # Should be same as Security group used for FSx. sg-03ZZZZZZ
job_name = "smdataparallel-efficientnet-pt-fsx-2p4d"  # This job name is used as prefix to the sagemaker training job. Makes it easy for your look for your training job in SageMaker Training job console.
file_system_id = "<fsx_id>"  # FSx file system ID with your training dataset. Example: 'fs-0bYYYYYY'

In [ ]:
# Configure the hyper-parameters
hyperparameters = {
    "arch": "efficientnet-b4",
    "batch-size": 16,
    "optimizer-batch-size": 4096,
    "lr": 0.16,
    "weight-decay": 5e-06,
    "image-size": 380,
    "optimizer": "rmsprop",
    "rmsprop-alpha": 0.9,
    "rmsprop-eps": 0.01,
    "print-freq": 100,
    "label-smoothing": 0.1,
    "mixup": 0.2,
    "lr-schedule": "cosine",
    "momentum": 0.9,
    "warmup": 16,
    "epochs": 5,
    "data-backend": "pytorch",
    "augmentation": "autoaugment",
    "num-classes": 1000,
    "interpolation": "bicubic"
}

In [ ]:
# Configure metrics to be displayed for the training job
# In this example, we show how to record a custom training throughput metric
# Reference: https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html
metric_definitions = [
    {"Name": "train_throughput", "Regex": "examples/second : (.*?) "},
]

In [ ]:
estimator = PyTorch(
    entry_point="entry_point.py",
    role=role,
    image_uri=docker_image,
    source_dir=".",
    instance_count=instance_count,
    instance_type=instance_type,
    framework_version="1.8.1",
    py_version="py38",
    sagemaker_session=sagemaker_session,
    hyperparameters=hyperparameters,
    subnets=subnets,
    security_group_ids=security_group_ids,
    debugger_hook_config=False,
    # Training using SMDataParallel Distributed Training Framework
    distribution={"smdistributed": {"dataparallel": {"enabled": True}}},
)

In [ ]:
# Configure FSx Input for your SageMaker Training job

from sagemaker.inputs import FileSystemInput

# YOUR_MOUNT_PATH_FOR_TRAINING_DATA # NOTE: '/fsx_mount_name/' will be the root mount path. Example: '/fsx_mount_name/imagenet''''
file_system_directory_path = "/<mount_name>"
file_system_access_mode = "rw"
file_system_type = "FSxLustre"
train_fs = FileSystemInput(
    file_system_id=file_system_id,
    file_system_type=file_system_type,
    directory_path=file_system_directory_path,
    file_system_access_mode=file_system_access_mode,
)
data_channels = {"train": train_fs}

In [ ]:
# Submit SageMaker training job
estimator.fit(inputs=data_channels, job_name=job_name)

## Next steps

Now that you have trained a model using Amazon SageMaker's distributed library, you can deploy an endpoint to host the model. After you deploy the endpoint, you can then test it with inference requests by following this [blog post](https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/). The following cell will store the model_data variable to be used for inference.

In [ ]:
model_data = estimator.model_data
print("Storing {} as model_data".format(model_data))
%store model_data

## Clean Up

To avoid incurring unnecessary charges, follow these [steps to use the AWS Management Console to delete resources such as endpoints, notebook instances, S3 buckets, and CloudWatch logs](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html).